In [1]:
using DataFrames, CSV, DelimitedFiles, JuMP, Gurobi
const GRB_ENV = Gurobi.Env()

Academic license - for non-commercial use only


Gurobi.Env(Ptr{Nothing} @0x0000000067d3a700, false, 0)

In [2]:
NUM_CREWS = 10                
BREAK_LENGTH = 2       # how long at base to be considered "rested"

# tradeoffs
BETA = 100             # cost of one area unit burned / cost of mile traveled
ALPHA = 200            # cost of crew-day of suppression / cost of mile traveled
LINE_PER_CREW = 17     # how much perimeter prevented per crew per time period

FIRE_CODE = 1
BASE_CODE = 2

2

In [3]:
struct GlobalData
    
    ff_dist::Matrix{Float64}
    bf_dist::Matrix{Float64}
    ff_tau::Matrix{Int64}
    bf_tau::Matrix{Int64}
    
end

struct CrewStatus
    
    rest_by::Vector{Int64}
    current_fire::Vector{Int64}
    rested_periods::Vector{Int64}
    
end

struct RegionData
    
    crew_regions::Vector{Int64}
    fire_regions::Vector{Int64}
    
end


struct KeyArcIndices
    
    # fire flow data
    f_out::Array{Vector{Int64}}
    f_in::Array{Vector{Int64}}
    
    # base flow data
    b_out::Array{Vector{Int64}}
    b_in::Array{Vector{Int64}}
    
    # total crews suppressing each fire
    supp_fire::Array{Vector{Int64}}
    
    # start constraints
    start::Array{Vector{Int64}}
    
    # assignments out of region
    out_of_region::Array{Vector{Int64}}
    
end 

mutable struct RouteData
    
    routes_per_crew::Vector{Int64} # could add in length
    route_costs::Matrix{Float64}
    fires_fought::BitArray{4}
    out_of_reg::BitArray{3}
    
end

mutable struct SuppressionPlanData
    
    plans_per_fire::Vector{Int64} # could add in length
    plan_costs::Matrix{Float64}
    crews_present::Array{Int8, 3}
    
end



In [4]:
struct FireConstraintIndices
    
    # fire flow data
    flow_out::Array{Vector{Int64}}
    flow_in::Array{Vector{Int64}}
    
    # total crews suppressing each fire
    crews_needed::Array{Vector{Int64}}
    
    # start constraints
    start::Array{Vector{Int64}}
    
end

In [5]:
struct ColumnGeneration
    
    route_sps::Vector{Any}
    plan_sps::Vector{Any}
    routes::RouteData
    suppression_plans::SuppressionPlanData
    
end

In [6]:
function get_rotation_orders(crew_regions)
    
    # initialize output
    out = Dict()
    
    # get the unique regions where there are crews
    regions = unique(crew_regions)
    
    # for each region
    for region in regions
        
        # initialize dictionary corresponding to the order
        out[region] = Dict() 
        crews_in_region = 0
        
        # for each crew in the region
        for crew in 1:NUM_CREWS
            
            if crew_regions[crew] == region
                
                # update crew count, log rotation order 
                crews_in_region += 1
                out[region][crew] = crews_in_region
            end
        end
    end
    
    return out
end

get_rotation_orders (generic function with 1 method)

In [7]:
# crew, from_type, from_ix, to_type, to_ix, from_time, to_time, from_rested, to_rested, exited_region

In [8]:
function arc_exits_region(crew, from_type, from_ix, to_type, to_ix, region_data)
    
    # get the region where the arc originates
    from_region = 0
    if from_type == FIRE_CODE
        from_region = region_data.fire_regions[from_ix]
    elseif from_type == BASE_CODE
        from_region = region_data.crew_regions[from_ix]
    else
        throw(DomainError(from_type, "from_type invalid"))
    end
    
    # get the region where the arc terminates
    to_region = 0
    if to_type == FIRE_CODE
        to_region = region_data.fire_regions[to_ix]
    elseif to_type == BASE_CODE
        to_region = region_data.crew_regions[to_ix]
    else
        throw(DomainError(from_type, "to_type invalid"))
    end
    
    # if these are different regions
    if from_region != to_region
        
        # if the crew is leaving its home region
        if region_data.crew_regions[crew] == from_region
        
            # return the region that the arc exited
            return from_region
        
        end
        
    end
    
    # otherwise
    return 0
    
end     

arc_exits_region (generic function with 1 method)

In [9]:
function generate_arcs(gd, rd, cs)
    
    # get fire-to-fire arcs
    ff = [[c, FIRE_CODE, f_from, FIRE_CODE, f_to, t_from, t_from + gd.ff_tau[f_to, f_from], rest, rest]
          for c=1:NUM_CREWS, f_from=1:NUM_FIRES, f_to=1:NUM_FIRES, t_from=1:NUM_TIME_PERIODS, rest=0:1]
    ff = copy(reduce(hcat, ff)')

    # get fire-to-fire arcs from start, based on cs.current crew locations
    from_start_ff = [[c, FIRE_CODE, cs.current_fire[c], FIRE_CODE, f_to, 0, gd.ff_tau[f_to, cs.current_fire[c]], 0, 0]
                      for c=1:NUM_CREWS, f_to=1:NUM_FIRES if cs.current_fire[c] != -1]
    from_start_ff = copy(reduce(hcat, from_start_ff)')

    # get base-to-fire arcs
    rf = [[c, BASE_CODE, c, FIRE_CODE, f_to, t_from, t_from + gd.bf_tau[c, f_to], rest, rest]
           for c=1:NUM_CREWS, f_to=1:NUM_FIRES, t_from=1:NUM_TIME_PERIODS, rest=0:1]
    rf = copy(reduce(hcat, rf)')

    # get base-to-fire arcs from start
    from_start_rf = [[c, BASE_CODE, c, FIRE_CODE, f_to, 0, gd.bf_tau[c, f_to], 0, 0]
                      for c=1:NUM_CREWS, f_to=1:NUM_FIRES if cs.current_fire[c] == -1]
    from_start_rf = copy(reduce(hcat, from_start_rf)')

    # get fire-to-base arcs
    fr = [[c, FIRE_CODE, f_from, BASE_CODE, c, t_from, t_from + gd.bf_tau[c, f_from], rest, rest]
           for c=1:NUM_CREWS, f_from=1:NUM_FIRES, t_from=1:NUM_TIME_PERIODS, rest=0:1]
    fr = copy(reduce(hcat, fr)')

    # get fire-to-base arcs from start, based on cs.current crew locations
    from_start_fr = [[c, FIRE_CODE, cs.current_fire[c], BASE_CODE, c, 0, gd.bf_tau[c, cs.current_fire[c]], 0, 0]
                      for c=1:NUM_CREWS if cs.current_fire[c] != -1]
    from_start_fr = copy(reduce(hcat, from_start_fr)')

    # get base-to-base arcs
    rr = [[c, BASE_CODE, c, BASE_CODE, c, t_from, t_from + 1 + (BREAK_LENGTH - 1) * rest, 0, rest]
          for c=1:NUM_CREWS, t_from=1:NUM_TIME_PERIODS, rest=0:1]
    rr = copy(reduce(hcat, rr)')
    rr_rested = [[c, BASE_CODE, c, BASE_CODE, c, t_from, t_from + 1, 1, 1]
          for c=1:NUM_CREWS, t_from=1:NUM_TIME_PERIODS]
    rr_rested  = copy(reduce(hcat, rr_rested)')

    # get base-to-base arcs from start, based on cs.current days rested
    from_start_rr = [[c, BASE_CODE, c, BASE_CODE, c, 0, 
                      1 + (BREAK_LENGTH - max(cs.rested_periods[c], 0) - 1) * rest, 0, rest] 
                      for c=1:NUM_CREWS, rest=0:1 if cs.current_fire[c] == -1]
    from_start_rr = copy(reduce(hcat, from_start_rr)')

    A = vcat(ff, from_start_ff, rf, from_start_rf, fr, from_start_fr, rr, rr_rested, from_start_rr)

    out_of_region = [arc_exits_region(A[i, 1], A[i, 2], A[i, 3], A[i, 4], A[i, 5], rd) 
                     for i in 1:length(A[:, 1])]
    A = hcat(A, out_of_region)
    
    return A
end

generate_arcs (generic function with 1 method)

In [10]:
function get_distance(from_type, from_ix, to_type, to_ix, fire_fire, base_fire)
    
    dist = 0
    
    # if fire to fire
    if (from_type == FIRE_CODE) & (to_type == FIRE_CODE)
        dist = fire_fire[from_ix, to_ix]
    
    # if fire to base
    elseif (from_type == FIRE_CODE) & (to_type == BASE_CODE)
        dist = base_fire[to_ix, from_ix]
    
    # if base to fire
    elseif (from_type == BASE_CODE) & (to_type == FIRE_CODE)
        dist = base_fire[from_ix, to_ix]
        
    # otherwise dist still 0
    end
    
    return dist
end 

get_distance (generic function with 1 method)

In [11]:
function get_arc_costs(gd, arcs, cost_param_dict)
    
    # get number of arcs
    n_arcs = length(arcs[:, 1])
    
    # initialize costs to 0
    costs = zeros(n_arcs)
    
    # if there is travel cost per mile
    if "cost_per_mile" in keys(cost_param_dict)
        
        # find the miles for each arc
        miles_per_arc =  [get_distance(arcs[i, 2], arcs[i, 3], 
                                       arcs[i, 4], arcs[i, 5], 
                                       gd.ff_dist, gd.bf_dist) for i in 1:n_arcs]
        # add to costs
        costs = costs .+ (cost_param_dict["cost_per_mile"] * miles_per_arc)
    end
    
    # if there are rest violations
    if "rest_violation" in keys(cost_param_dict)
        
        # find the rest violation scores
        rest_violation_matrix = cost_param_dict["rest_violation"]
        rest_violations = [(arcs[i, 8] == 0) & (arcs[i, 6] > 0) ? 
                           rest_violation_matrix[arcs[i, 1], arcs[i, 6]] : 0
                           for i in 1:n_arcs]
        
        # add to costs
        costs = costs .+ rest_violations
    end
    
    if "fight_fire" in keys(cost_param_dict)
        costs = costs .+ [(arcs[i, 4] == FIRE_CODE) ? cost_param_dict["fight_fire"] : 0
                          for i in 1:n_arcs]
    end
    
    # if we have to adjust for linking dual constraints
    if "linking_dual" in keys(cost_param_dict)
        
        # get the dual variables
        rho = cost_param_dict["linking_dual"]
        
        # get linking costs (really benefits) if arc goes to a fire
        linking_costs = [((arcs[i, 4] == FIRE_CODE) & (arcs[i, 7] <= NUM_TIME_PERIODS)) ? 
                          - rho[arcs[i, 5], arcs[i, 7]] : 0
                          for i in 1:n_arcs]
        
        # add to costs
        costs = costs .+ linking_costs
        
    end
    
    # if we have to adjust for linking dual constraints
    if "out_of_region_dual" in keys(cost_param_dict)
        
        # get needed regional info
        regs = cost_param_dict["region_data"].crew_regions
        rot_order = cost_param_dict["rotation_order"]
        
        # get the dual variables
        eta = cost_param_dict["out_of_region_dual"]

        # get adjustment for crew allotment
        c1 = [(arcs[i, 10] > 0) ? sum(eta[arcs[i, 1], t_0]
                                        for t_0=arcs[i, 6]:NUM_TIME_PERIODS
                                      ) : 0
                                                   
               for i in 1:n_arcs
             ]
        
        # get adjustment for region average allotment
        c2 = [(arcs[i, 10] > 0) ? sum(eta[c, t_0]
                                            for c in keys(rot_order[regs[arcs[i, 1]]]),
                                                t_0=arcs[i, 6]:NUM_TIME_PERIODS) /
                                        length(keys(rot_order[regs[arcs[i, 1]]])) : 0
                                                   
               for i in 1:n_arcs
             ]
        
        # get adjustment for big-M constraint
        c3 = [(arcs[i, 10] > 0) ? NUM_TIME_PERIODS * eta[arcs[i, 1], arcs[i, 6]] : 0
               for i in 1:n_arcs
             ]
            
        # add to costs
        costs = costs .+ c1 .- c2 .+ c3
        
    end   
    
    return costs
end

get_arc_costs (generic function with 1 method)

In [12]:
function positive(x)
    
    if x > 0
        return 1
    end
    
    return 0
end

function is_one(x)
    
    if x == 1
        return 1
    end
    
    return 0
end

is_one (generic function with 1 method)

In [13]:
# should return matrix indexed by crew, time, 
function get_rest_penalties(rest_by_periods, lambda, accounting_func)
    
    penalties = zeros(NUM_CREWS, NUM_TIME_PERIODS)
    
    for c in 1:NUM_CREWS
        penalties[c, :] = [lambda * accounting_func(t - rest_by_periods[c]) 
                           for t in 1:NUM_TIME_PERIODS]
    end
    
    return penalties    
end

get_rest_penalties (generic function with 1 method)

In [14]:
function define_network_constraint_data(arcs)
    
    # shorten some global variable names
    C = NUM_CREWS
    G = NUM_FIRES
    T = NUM_TIME_PERIODS
    
    # get number of arcs
    n_arcs = length(arcs[:, 1])
      
    ## flow balance ##
    
    # initialize arrays of vectors for flow balance
    f_out = Array{Vector{Int64}}(undef, C, G, T, 2)
    f_in = Array{Vector{Int64}}(undef, C, G, T, 2)
    b_out = Array{Vector{Int64}}(undef, C, T, 2)
    b_in = Array{Vector{Int64}}(undef, C, T, 2)
    start = Array{Vector{Int64}}(undef, C)
    out_of_region = Array{Vector{Int64}}(undef, C, T+1)
    
    # for each crew
    for crew in 1:C
        
        # get indices of this crew's arcs only
        crew_ixs = [i for i in 1:n_arcs if arcs[i, 1] == crew]
        
        # get time 0 indices
        start[crew] = [i for i in crew_ixs if arcs[i, 6] == 0]
        
        # for each time period (including start)
        for tm in 0:T
        
            # get indices for out of region assignments
            out_of_region[crew, tm+1] = [i for i in crew_ixs if
                                           (arcs[i, 6] == tm) &
                                           (arcs[i, 10] > 0)
                                        ]
        end
        
        # for each time period
        for tm in 1:T
            
            # for each rest state
            for rest in 1:2
                
                # get arcs leaving crew base at this time with this rest
                b_out[crew, tm, rest] = [i for i in crew_ixs if
                                         (arcs[i, 2] == BASE_CODE) &
                                         (arcs[i, 6] == tm) &
                                         (arcs[i, 8] == rest-1)
                                        ]
                
                # get arcs entering crew base at this time with this rest
                b_in[crew, tm, rest] = [i for i in crew_ixs if
                                        (arcs[i, 4] == BASE_CODE) &
                                        (arcs[i, 7] == tm) &
                                        (arcs[i, 9] == rest-1)
                                       ]
                # for each fire
                for fire in 1:G
                    
                    # get arcs where this crew leaves this fire at this time
                    # with this rest state
                    f_out[crew, fire, tm, rest] = [i for i in crew_ixs if
                                                   (arcs[i, 2] == FIRE_CODE) &
                                                   (arcs[i, 3] == fire) &
                                                   (arcs[i, 6] == tm) &
                                                   (arcs[i, 8] == rest-1)
                                                   ]
                    
                    # get arcs where this crew enters this fire at this time
                    # with this rest state
                    f_in[crew, fire, tm, rest] = [i for i in crew_ixs if
                                                  (arcs[i, 4] == FIRE_CODE) &
                                                  (arcs[i, 5] == fire) &
                                                  (arcs[i, 7] == tm) &
                                                  (arcs[i, 9] == rest-1)
                                                  ]
                end
            end
        end
    end
    
    ## linking constraints ##
    linking = Array{Vector{Int64}}(undef, G, T)
    for fire in 1:G
        for tm in 1:T
            
            # we count the crew as working *where they arrived* during this timestep
            linking[fire, tm] = [i for i in 1:n_arcs if (arcs[i, 4] == FIRE_CODE) &
                                                        (arcs[i, 5] == fire) &
                                                        (arcs[i, 7] == tm)]
        end
    end
    
    
    return KeyArcIndices(f_out, f_in, b_out, b_in, linking, start, out_of_region)
end

define_network_constraint_data (generic function with 1 method)

In [15]:
function get_route_stats(arc_ixs_used, arcs, costs)
    
    # get total cost
    route_cost = sum(costs[arc_ixs_used])
    
    # initialize fires fought matrix
    fires_fought =  falses(NUM_FIRES, NUM_TIME_PERIODS)
    
    # initialize out of region matrix
    out_of_region = falses(NUM_TIME_PERIODS + 1)
    
    # for each arc used
    for ix in arc_ixs_used
        arc = arcs[ix, :]
        
        # update fires_fought
        if (arc[4] == FIRE_CODE) & (arc[7] <= NUM_TIME_PERIODS)
            @assert ~fires_fought[arc[5], arc[7]] "Visited fire twice at same time"
            fires_fought[arc[5], arc[7]] = true
        end
        
        # update out_of_region
        if arc[10] > 0
            @assert ~out_of_region[arc[6] + 1] "Left region twice at same time"
            out_of_region[arc[6] + 1] = true
        end
    end
    
    return route_cost, fires_fought, out_of_region
end

get_route_stats (generic function with 1 method)

In [16]:
function initialize_route_data(max_routes)
    
    return RouteData(zeros(NUM_CREWS), Matrix{Float64}(undef, NUM_CREWS, max_routes),
                     BitArray(undef, NUM_CREWS, max_routes, NUM_FIRES, NUM_TIME_PERIODS) .> 2,
                     BitArray(undef, NUM_CREWS, max_routes, NUM_TIME_PERIODS + 1) .> 2)
end

initialize_route_data (generic function with 1 method)

In [17]:
function update_available_routes(crew, route_ixs, arcs, costs, route_data)
    
    # get the required information from the arcs used
    route_cost, fires_fought, out_of_region = get_route_stats(route_ixs, arcs, costs)
    
    ## store this information to the route_data ##
    
    # add 1 to number of routes for this crew, store the index
    route_data.routes_per_crew[crew] += 1
    ix = route_data.routes_per_crew[crew]
    
    # append the route cost
    route_data.route_costs[crew, ix] = route_cost
    
    # append the fires fought
    route_data.fires_fought[crew, ix, :, :] = fires_fought
    
    # append the out-of-region assignments
    route_data.out_of_reg[crew, ix, :] = out_of_region
    
    return 1

end

update_available_routes (generic function with 1 method)

In [18]:
function get_supp_plan_stats(var_p, var_d, beta, tolerance=0.0001)
    
    # get the cost based on the perimeter progression
    cost = beta * (sum(value.(var_p)) - value(var_p[1])/2 - value(var_p[NUM_TIME_PERIODS+1]/2))
    
    # get the number of crews present each time period from line constructed
    crew_vector = value.(var_d)
    int_crew_vector = convert.(Int64, round.(crew_vector))
    @assert maximum(abs.(crew_vector - int_crew_vector)) < tolerance "Not an integer plan"
    
    return cost, int_crew_vector

end

get_supp_plan_stats (generic function with 2 methods)

In [19]:
function initialize_supp_plan_data(max_supp_plans)
    
    return SuppressionPlanData(zeros(NUM_FIRES), 
                               Matrix{Float64}(undef, NUM_FIRES, max_supp_plans),
                               zeros(Int8, (NUM_FIRES, max_supp_plans, NUM_TIME_PERIODS))
                              )
end

initialize_supp_plan_data (generic function with 1 method)

In [20]:
function update_available_supp_plans(fire, p, d, beta, plan_data)
    
    # get the required information from the model decision variables
    cost, crew_vector = get_supp_plan_stats(p, d, beta)
    
    ## store this information to the suppression plan data ##
    
    # add 1 to number of plans for this fire, store the index
    plan_data.plans_per_fire[fire] += 1
    ix = plan_data.plans_per_fire[fire]
    
    # append the route cost
    plan_data.plan_costs[fire, ix] = cost
    
    # append the fires fought
    plan_data.crews_present[fire, ix, :] = crew_vector
    
    return 1

end

update_available_supp_plans (generic function with 1 method)

In [21]:
function full_formulation(integer_routes, region_data, constraint_data, rotation_order, 
                          costs, progs, perims, beta, gamma, verbose=false)
    
    # get number of arcs
    n_arcs = length(costs)
    
    # shorten some global variable names
    C = NUM_CREWS
    G = NUM_FIRES
    T = NUM_TIME_PERIODS
    regs = region_data.crew_regions
    
    # intialize model
    m = Model(() -> Gurobi.Optimizer(GRB_ENV))
    
    if ~verbose
        set_optimizer_attribute(m, "OutputFlag", 0)
    end

    # fire suppression plan section
    @variable(m, p[g=1:G, t=1:T+1] >= 0)
    @variable(m, l[g=1:G, t=1:T])
    @variable(m, d[g=1:G, t=1:T] >= 0)
    @constraint(m, perim_growth[g=1:G, t=1:T], p[g, t+1] >= progs[g, t] * 
                                                           (p[g, t] - l[g, t] / 2) - l[g, t] / 2)
    @constraint(m, perim_start[g=1:G], p[g, 1] == perims[g])

    # routing plan section
    if integer_routes
        @variable(m, z[1:n_arcs] >= 0, Int)
    else
        @variable(m, z[1:n_arcs] >= 0)
    end
    
    @variable(m, q[1:C, 0:T] >= 0, Int)
    
    # build out_of_region constraints
    @constraint(m, out_of_region[c=1:C, t=0:T],
    
        # out of region penalty is at least
        q[c, t] >=
        
            # this crew's cumulative rotations
            sum(z[i] for t_0=0:t, i in constraint_data.out_of_region[c, t_0+1]) 
        
        - 
        
            # average cumulative rotations among all crews in same region
            sum(z[i] for c_0 in keys(rotation_order[regs[c]]), t_0=0:t, 
                i in constraint_data.out_of_region[c_0, t_0+1]) /
            length(keys(rotation_order[regs[c]]))
        
        -
        
            # normalizing factor for specific crew rotation order
            (1 - rotation_order[regs[c]][c] / length(keys(rotation_order[regs[c]])))
        
        -
            # big-M for if crew goes not leave region at this time
            T * (1 - sum(z[i] for i in constraint_data.out_of_region[c, t+1]))
        
    )


    @constraint(m, fire_flow[c=1:C, g=1:G, t=1:T, rest=1:2],

            sum(z[constraint_data.f_out[c, g, t, rest]]) ==
            sum(z[constraint_data.f_in[c, g, t, rest]])
    
    )
    
    @constraint(m, base_flow[c=1:C, t=1:T, rest=1:2],

            sum(z[constraint_data.b_out[c, t, rest]]) ==
            sum(z[constraint_data.b_in[c, t, rest]])
    
    )


    @constraint(m, linking[g=1:G, t=1:T],

        sum(z[constraint_data.supp_fire[g, t]]) >= d[g, t] 
    )
    
    @constraint(m, line_building[g=1:G, t=1:T], l[g, t] <= LINE_PER_CREW * d[g, t])

    # build start constraint
    @constraint(m, start[c=1:C], 

        sum(z[constraint_data.start[c]]) == 1
    )
    
    
    

    @objective(m, Min, 
        beta * (sum(p) - sum(p[1:G, 1])/2 - sum(p[1:G, T+1])/2) + 
        sum(z .* costs) + sum(q) * gamma
    )
    
    return m, p, d, z, q, out_of_region
    
end

full_formulation (generic function with 2 methods)

In [22]:
function load_data(path)
    
    # get distance from fire f to fire g 
    fire_dists =  readdlm(path * "/fire_distances.csv", ',')

    # get distance from base c to fire g (NUM_CREWS-by-NUM_FIRES)
    base_fire_dists =  readdlm(path * "/base_fire_distances.csv", ',')

    # initialize travel times (number of periods) from fire f to fire g
    tau = convert(Array{Int}, ones(size(fire_dists)))

    # initialize number of periods to travel from base c to fire g (NUM_CREWS-by-NUM_FIRES)
    tau_base_to_fire = convert(Array{Int}, ones((size(base_fire_dists))))

    # read intial crew statuses (location, period by which they must rest)
    # (-1 in current_fire means crew is currently at base)
    # (rested_periods is the amount of time crew has been at base, relevant for completing rest)
    crew_starts = CSV.read(path * "/sample_crew_starts.csv", DataFrame)
    rest_by = crew_starts[!, "rest_by"]
    current_fire = crew_starts[!, "current_fire"]
    rested_periods = crew_starts[!, "rested_periods"]


    return (GlobalData(fire_dists, base_fire_dists, tau, tau_base_to_fire), 
            CrewStatus(rest_by, current_fire, rested_periods))
end

load_data (generic function with 1 method)

In [23]:
function master_problem(route_data, supp_plan_data, region_data, rotation_order, gamma, price_branch=false)
    
    m = Model(() -> Gurobi.Optimizer(GRB_ENV))
    set_optimizer_attribute(m, "OutputFlag", 0)
    
    regs = region_data.crew_regions
    
    # decision variables
    if price_branch
        @variable(m, route[c=1:NUM_CREWS, r=1:route_data.routes_per_crew[c]] >= 0, Int)
        @variable(m, plan[g=1:NUM_FIRES, p=1:supp_plan_data.plans_per_fire[g]] >= 0, Int)
        @variable(m, q[c=1:NUM_CREWS, t=0:NUM_TIME_PERIODS] >= 0, Int)
    else
        @variable(m, route[c=1:NUM_CREWS, r=1:route_data.routes_per_crew[c]] >= 0)
        @variable(m, plan[g=1:NUM_FIRES, p=1:supp_plan_data.plans_per_fire[g]] >= 0)
        @variable(m, q[c=1:NUM_CREWS, t=0:NUM_TIME_PERIODS] >= 0)
    end
    
    # constraints that you must choose a plan per crew and per fire
    @constraint(m, route_per_crew[c=1:NUM_CREWS], 
                sum(route[c, r] for r=1:route_data.routes_per_crew[c]) == 1)
    @constraint(m, plan_per_fire[g=1:NUM_FIRES], 
                sum(plan[g, p] for p=1:supp_plan_data.plans_per_fire[g]) >= 1)
    
    # linking constraint
    @constraint(m, linking[g=1:NUM_FIRES, t=1:NUM_TIME_PERIODS],
                    
                    # crews at fire
                    sum(route[c, r] * route_data.fires_fought[c, r, g, t] 
                        for c=1:NUM_CREWS, r=1:route_data.routes_per_crew[c]) 
        
                    >=
        
                    # crews suppressing
                    sum(plan[g, p] * supp_plan_data.crews_present[g, p, t] 
                        for p=1:supp_plan_data.plans_per_fire[g]) 
        
                )
    
    # out_of_region constraint
    @constraint(m, out_of_region[c=1:NUM_CREWS, t=0:NUM_TIME_PERIODS],
    
        # out of region penalty is at least
        q[c, t] >=
        
            # this crew's cumulative rotations
            sum(route[c, r] * route_data.out_of_reg[c, r, t_0 + 1] 
            for r=1:route_data.routes_per_crew[c], t_0=0:t)
        
        - 
        
            # average cumulative rotations among all crews in same region
            sum(route[c_0, r] * route_data.out_of_reg[c_0, r, t_0 + 1] 
                for c_0 in keys(rotation_order[regs[c]]), r=1:route_data.routes_per_crew[c_0],
                t_0=0:t) /
            length(keys(rotation_order[regs[c]]))
        
        -
        
            # normalizing factor for specific crew rotation order
            (1 - rotation_order[regs[c]][c] / length(keys(rotation_order[regs[c]])))
        
        -
            # big-M for if crew goes not leave region at this time
            NUM_TIME_PERIODS * (1 - sum(route[c, r] * route_data.out_of_reg[c, r, t+1]
                                        for r=1:route_data.routes_per_crew[c])
                               )
        
    )
    
    @objective(m, Min, 
        
                  # route costs
                  sum(route[c, r] * route_data.route_costs[c, r] 
                        for c=1:NUM_CREWS, r=1:route_data.routes_per_crew[c])
        
                  +
                     
                  # suppression plan costs
                  sum(plan[g, p] * supp_plan_data.plan_costs[g, p] 
                     for g=1:NUM_FIRES, p=1:supp_plan_data.plans_per_fire[g]) 
        
                  +
        
                  # rotational queueing violations cost
                  sum(q) * gamma
               )
    
    return Dict("m" => m, "q" => q, "sigma" => route_per_crew, "pi" => plan_per_fire, 
                "rho" => linking, "eta" => out_of_region, "route" => route, "plan" => plan)
end 

master_problem (generic function with 2 methods)

In [24]:
function init_route_subproblem(crew_ixs, crew, constraint_data, integer_routes=false)
    
    # shorten some global variable names
    C = NUM_CREWS
    G = NUM_FIRES
    T = NUM_TIME_PERIODS
    
    # intialize model
    m = Model(() -> Gurobi.Optimizer(GRB_ENV))
    set_optimizer_attribute(m, "OutputFlag", 0)

    # routing plan section
    if integer_routes
        @variable(m, z[crew_ixs] >= 0, Int)
    else
        @variable(m, z[crew_ixs] >= 0)
    end


    @constraint(m, fire_flow[g=1:G, t=1:T, rest=1:2],

            sum(z[constraint_data.f_out[crew, g, t, rest]]) ==
            sum(z[constraint_data.f_in[crew, g, t, rest]])
    
    )
    
    @constraint(m, base_flow[t=1:T, rest=1:2],

            sum(z[constraint_data.b_out[crew, t, rest]]) ==
            sum(z[constraint_data.b_in[crew, t, rest]])
    
    )

    # build start constraint
    @constraint(m, start, 

        sum(z[constraint_data.start[crew]]) == 1
    )
    
    return Dict("m" => m, "z" => z, "ff" => fire_flow)
    
end

init_route_subproblem (generic function with 2 methods)

In [25]:
function init_suppression_plan_subproblem(progs, perims, fire, beta)
    
    T = NUM_TIME_PERIODS
    
    m = Model(() -> Gurobi.Optimizer(GRB_ENV))
    set_optimizer_attribute(m, "OutputFlag", 0)

    # fire suppression plan section
    @variable(m, p[t=1:T+1] >= 0)
    @variable(m, l[t=1:T] >= 0)
    @variable(m, NUM_CREWS >= d[t=1:T] >= 0, Int)
    @constraint(m, suppression_per_crew[t=1:T], l[t] <= d[t] * LINE_PER_CREW)
    @constraint(m, perim_growth[t=1:T], p[t+1] >= progs[fire, t] * (p[t] - l[t] / 2) - l[t] / 2)
    @constraint(m, perim_start, p[1] == perims[fire])
    
#    
    
    return Dict("m" => m, "p" => p, "d" => d, "beta" => beta)
end

init_suppression_plan_subproblem (generic function with 1 method)

In [26]:
function initialize_column_generation(arcs, costs, constraint_data, progs, perims, max_plans)
    
    # initialize subproblems
    route_sps = []
    for crew in 1:NUM_CREWS
        ixs = [i for i in 1:length(arcs[:, 1]) if arcs[i, 1] == crew]
        d = init_route_subproblem(ixs, crew, constraint_data)
        d["arc_ixs"] = ixs
        push!(route_sps, d)
    end

    plan_sps = []
    for fire in 1:NUM_FIRES
        d = init_suppression_plan_subproblem(progs, perims, fire, BETA)
        push!(plan_sps, d)
    end
    
    # initialize routes and suppression plans to populate
    routes = initialize_route_data(max_plans)
    suppression_plans = initialize_supp_plan_data(max_plans)
    
    ## generate dummy plans (no suppression) to ensure feasibility at first step ##
    
    # for each crew
    for crew in 1:NUM_CREWS
        
        # get the crew's subproblem instance
        crew_sp = route_sps[crew]
        m = crew_sp["m"]
        z = crew_sp["z"]
        crew_ixs = crew_sp["arc_ixs"]

        # set objective in light of dual variables
        @objective(m, Min, sum(z[ix] * (costs[ix]) for ix in crew_ixs))

        # optimize
        optimize!(m)

        # update crew routes
        crew_arcs = [i for i in crew_ixs if (value(z[i]) > 0.5)]
        update_available_routes(crew, crew_arcs, arcs, costs, routes)
    
    end
    
    # for each fire
    for fire in 1:NUM_FIRES

        # get the fire's subproblem instance
        plan_sp = plan_sps[fire]
        m = plan_sp["m"]
        p = plan_sp["p"]
        d = plan_sp["d"]
        beta = plan_sp["beta"]

        # set objective in light of dual variables
        @objective(m, Min, sum(d))
        optimize!(m)
        
        # update suppression plans
        update_available_supp_plans(fire, p, d, beta, suppression_plans)

    end
    
    return ColumnGeneration(route_sps, plan_sps, routes, suppression_plans)
    
end

initialize_column_generation (generic function with 1 method)

In [27]:
function run_crew_subproblem(sps, crew, costs, local_costs)
    
    # get the crew's subproblem instance
    crew_sp = sps[crew]
    m = crew_sp["m"]
    z = crew_sp["z"]
    crew_ixs = crew_sp["arc_ixs"]
    
    # set objective in light of dual variables
    @objective(m, Min, sum(z[ix] * (local_costs[ix] + costs[ix]) for ix in crew_ixs))
        
    # optimize
    optimize!(m)
    
    return objective_value(m), z
end

run_crew_subproblem (generic function with 1 method)

In [28]:
function run_CG_step(cg, arcs, costs, global_data, region_data, rot_order, gamma)
    
    # formulate and solve the master problem
    mp = master_problem(cg.routes, cg.suppression_plans, region_data, rot_order, gamma)
    optimize!(mp["m"])

    # grab the dual variables
    sigma = dual.(mp["sigma"])
    rho = dual.(mp["rho"])
    eta = dual.(mp["eta"])
    pie = dual.(mp["pi"]) # lol can't overwrite "pi" in Julia

    # using the dual variables, get the local adjustments to the arc costs in the route subproblems
    d = Dict("out_of_region_dual" => eta, "region_data"=> region_data, "rotation_order" => rot_order, "linking_dual" => rho)
    local_costs = get_arc_costs(global_data, arcs, d)

    ## run subproblems ##

    # for each fire
    for fire in 1:NUM_FIRES

        # run the subproblem
        plan_sp = cg.plan_sps[fire]
        m = plan_sp["m"]
        p = plan_sp["p"]
        d = plan_sp["d"]
        beta = plan_sp["beta"]
        @objective(m, Min, beta * (sum(p) - p[1]/2 - p[NUM_TIME_PERIODS + 1]/2) + sum(d .* rho[fire, :]) + 0.0001 * sum(d))
        optimize!(m)

        # if there is an improving plan
        if objective_value(m) < pie[fire]

            # add it
            update_available_supp_plans(fire, p, d, beta, cg.suppression_plans)

        end
    end

    # for each crew
    for crew in 1:NUM_CREWS

        # run the crew subproblem
        obj, assignments = run_crew_subproblem(cg.route_sps, crew, costs, local_costs)

        # if there is an improving route
        if obj < sigma[crew]

            # add it
            crew_arcs = [i for i in cg.route_sps[crew]["arc_ixs"] if (value(assignments[i]) > 0.5)]
            update_available_routes(crew, crew_arcs, arcs, costs, cg.routes)

        end

    end 
    return mp
end

run_CG_step (generic function with 1 method)

In [29]:
in_path = "data/processed"

# get inital fire perimeters and no-suppression progression parameters
M = readdlm(in_path * "/sample_growth_patterns.csv", ',')
start_perims = M[:, 1]
progressions = M[:, 2:15]

NUM_TIME_PERIODS = size(M)[2] - 1 
NUM_FIRES = size(M)[1]      

g_data, crew_status = load_data(in_path)
r_data = RegionData([1, 1, 1, 1, 1, 2, 2, 2, 2, 2], [1, 1, 2, 2])
rotation_order = get_rotation_orders(r_data.crew_regions)
A = generate_arcs(g_data, r_data, crew_status);

rest_pen = get_rest_penalties(crew_status.rest_by, 99999, positive)
cost_params = Dict("cost_per_mile"=> 1, "rest_violation" => rest_pen, "fight_fire" => ALPHA)
arc_costs = get_arc_costs(g_data, A, cost_params)

c_data = define_network_constraint_data(A);

In [ ]:
@variable(m, p[g=1:G, t=1:T+1] >= 0)
@variable(m, l[g=1:G, t=1:T])
@variable(m, d[g=1:G, t=1:T] >= 0)
@constraint(m, perim_growth[g=1:G, t=1:T], p[g, t+1] >= progs[g, t] * 
                                                       (p[g, t] - l[g, t] / 2) - l[g, t] / 2)
@constraint(m, perim_start[g=1:G], p[g, 1] == perims[g])

@objective(m, Min, beta * (sum(p) - p[1]/2 - p[NUM_TIME_PERIODS + 1]/2))

In [30]:
function update_fire_stats(curr_stats, curr_time, crew_allocation, params)
    
    if params["model_type"] == "simple_linear"
        
        line_per_crew = params["line_per_crew"]
        prog = params["progressions"][curr_time]
        line = line_per_crew * crew_allocation
        next_stats = (curr_stats - line/2) * prog - line/2
        next_stats = max(0, next_stats)
        
    else
        error("Not implemented")
    end
    
    return next_stats
end

update_fire_stats (generic function with 1 method)

In [31]:
function inverse_update_fire_stats(stats_from, stats_to, time_from, time_to, params)
    """ Returns number of crews needed for given fire transition """
    
    if params["model_type"] == "simple_linear"

        line_per_crew = params["line_per_crew"]
        prog = params["progressions"][time_from]
        
        crews = 2 / line_per_crew * (prog * stats_from - stats_to) / (1 + prog)
        
    else
        error("Not implemented")
    end
    
    return crews
    
end

inverse_update_fire_stats (generic function with 1 method)

In [112]:
function create_discrete_fire_states(params)
    
    if params["model_type"] == "simple_linear"
        
        # get the no-suppression progression of this fire
        progs = params["progressions"]
        start_perim = params["start_perim"]
        no_supp = [start_perim]
        for i in 1:NUM_TIME_PERIODS
            push!(no_supp, no_supp[i] * progs[i])
        end
        
        # generalize this later
        aggressive_precision = 1
        num_aggressive_states = convert(Int, round(start_perim * 2 / aggressive_precision))
        num_passive_states = 100

        aggressive_states = LinRange(0, num_aggressive_states * aggressive_precision, num_aggressive_states)
        passive_states = exp.(LinRange(log(num_aggressive_states * aggressive_precision + 1), 
                                       maximum(log.(no_supp .+ 1)), num_passive_states + 1)
                             )
        passive_states = passive_states[2:num_passive_states+1] .- 1
        all_states = vcat(aggressive_states, passive_states)
        all_states = vcat(all_states, 9999999)
        
        push!(all_states, start_perim)
        
    else
        error("Not implemented")
    end
    
    return all_states
end

create_discrete_fire_states (generic function with 1 method)

In [73]:
function generate_state_transition_crew_reqs(curr_stats, curr_time, sorted_states, params, round_types)
    
    if params["model_type"] == "simple_linear"
        
        # get all possibly feasible states
        min_state_val = update_fire_stats(curr_stats, curr_time, NUM_CREWS + 0.5, params)
        max_state_val = update_fire_stats(curr_stats, curr_time, 0, params)
        
        # get min and max index of the possibly feasible states
        min_state_ix = searchsorted(sorted_states, min_state_val).start
        max_state_ix = searchsorted(sorted_states, max_state_val).start
        
        # inititalize output 
        output = Dict()
        for round_type in round_types
            output[round_type] = []
        end
        
        # inititalize minimum crews needed so far, for trimming (explained below)
        min_used = Dict()
        for round_type in round_types
            min_used[round_type] = NUM_CREWS + 1
        end
        
        # for each feasible state
        for state_ix in min_state_ix:max_state_ix
            crews_needed = inverse_update_fire_stats(curr_stats, sorted_states[state_ix], curr_time, 
                                                     curr_time + 1, params)
            
            # for each round type
            for round_type in round_types
                
                # round the number of crews
                if round_type == "ceiling"
                    crews = max(0, convert(Int, ceil(crews_needed - 0.0001)))
                elseif round_type == "floor"
                    crews = max(0, convert(Int, floor(crews_needed + 0.0001)))
                elseif round_type == "nearest"
                    crews = max(0, convert(Int, round(crews_needed)))
                else
                    error("Round type not implemented")
                end
                
                # since the states are sorted in increasing level of cost and risk
                # we can trim arcs that are dominated
                
                # if this is a feasible number of crews and we do not have a dominating arc
                if (crews <= NUM_CREWS) & (crews < min_used[round_type])
                    
                    # update the minimum crews used for this round type
                    min_used[round_type] = crews
                    
                    # push the crew requirement for this transition to the corresponding list
                    push!(output[round_type], (state_ix, crews))
                end
            end
        end
        
        return output

    
    else
        error("Model type not implemented")
    end
    
end
        

generate_state_transition_crew_reqs (generic function with 1 method)

In [74]:
function generate_graphs(states, params, round_types)
    
    # separate out non start states
    non_start_states = states[1:length(states)-1]
    
    # initializations
    n_states = length(non_start_states)
    crews_needed = Dict()
    
    # for each round type
    for round_type in round_types
        
        # init graph as array of vectors (each index is a state*time vertex, each vector holds edges out)
        crews_needed[round_type] = Array{Vector{}}(undef, n_states + 1, NUM_TIME_PERIODS + 1)
        
        # initialize time, state indices to check next
        curr_time = 1
        next_to_check = [n_states + 1]
        
        # while we have not yet reached the end of the horizon
        while curr_time < NUM_TIME_PERIODS + 1
            
            # copy the indices to check and reset next_to_check
            to_check = copy(next_to_check)
            next_to_check = []

            # for each state index feasible to reach at this time period
            for check in to_check

                # if it is not the last (no return) state
                if check != n_states
                    
                    # find the crew requirements for feasible edges
                    edges = generate_state_transition_crew_reqs(states[check], curr_time, non_start_states, 
                                                                params, [round_type])[round_type]
                
                # otherwise stay at this state
                else
                    edges = [(n_states, 0)]
                end

                # update the crews needed for each edge coming out of this state
                crews_needed[round_type][check, curr_time] = edges

                # append the neighbors to next_to_check
                next_to_check = vcat(next_to_check, [edges[i][1] for i in 1:length(edges) 
                                                     if ~(edges[i][1] in next_to_check)])
            end
            
            # add one to the time
            curr_time += 1
        end
    end
    
    return crews_needed
end

generate_graphs (generic function with 1 method)

In [75]:
progressions, start_perims

([1.5 1.0 … 0.6 0.5; 1.0 1.0 … 2.0 2.0; 4.0 4.0 … 0.9 0.9], [1000.0, 500.0, 100.0])

In [118]:
d = Dict("model_type" => "simple_linear", "progressions" => progressions[1, :], 
        "start_perim" => 1000., "line_per_crew" => LINE_PER_CREW)

Dict{String, Any} with 4 entries:
  "model_type"    => "simple_linear"
  "line_per_crew" => 17
  "start_perim"   => 1000.0
  "progressions"  => [1.5, 1.0, 1.0, 0.9, 0.9, 0.9, 0.8, 0.8, 0.8, 0.7, 0.7, 0.…

In [119]:
@time states = create_discrete_fire_states(d);

  0.000240 seconds (101 allocations: 72.633 KiB)


In [120]:
@time f = generate_graphs(states, d, ["ceiling", "floor"])["floor"];

  1.608785 seconds (31.63 M allocations: 526.460 MiB, 7.06% gc time)


In [121]:
@time a = [(i, j) for i in 1:size(f)[1], j in 1:size(f)[2] if isassigned(f, i, j)]
@time sum(length(f[a1...]) for a1 in a)

  0.096059 seconds (131.37 k allocations: 6.277 MiB, 98.16% compilation time)
  0.031079 seconds (55.40 k allocations: 2.930 MiB, 95.92% compilation time)


46210

In [123]:
rho_test = ones(NUM_TIME_PERIODS) .* 5
@time avail_nodes = Dict(i => [j for j in 1:size(f)[1] if isassigned(f, j, i)] for i in 1:size(f)[2])
curr_time = NUM_TIME_PERIODS
costs = zeros(length(states), NUM_TIME_PERIODS + 1) .+ 1.0e12
costs[:, NUM_TIME_PERIODS + 1] = states / 2
bests = zeros(length(states), NUM_TIME_PERIODS) .- 1

@time while curr_time > 0
    
    state_costs = states
    if curr_time == 1
        state_costs = states/2
    end
    
    nodes_to_check = avail_nodes[curr_time]
    for node in nodes_to_check
        current_cost = 1e10
        current_best = -1
        edges = f[node, curr_time]
        for edge in edges
            edge_cost = costs[edge[1], curr_time + 1] + rho_test[curr_time] * edge[2]
            if edge_cost < current_cost
                current_best = edge[1]
                current_cost = edge_cost
            end
        end
        costs[node, curr_time] = current_cost + state_costs[node]
        bests[node, curr_time] = current_best
    end
    
    curr_time -= 1
end
    
    

  0.127661 seconds (236.09 k allocations: 12.734 MiB, 98.03% compilation time)
  0.038878 seconds (312.85 k allocations: 6.274 MiB)


In [124]:
costs

2102×15 Matrix{Float64}:
    1.0e12  1.0e12  1.0e12  1.0e12  1.0e12  …   0.0      0.0       0.0
    1.0e12  1.0e12  1.0e12  1.0e12  1.0e12      1.0005   1.0005    0.50025
    1.0e12  1.0e12  1.0e12  1.0e12  1.0e12      2.001    2.001     1.0005
    1.0e12  1.0e12  1.0e12  1.0e12  1.0e12      3.0015   3.0015    1.50075
    1.0e12  1.0e12  1.0e12  1.0e12  1.0e12      4.002    4.002     2.001
    1.0e12  1.0e12  1.0e12  1.0e12  1.0e12  …   5.0025   5.0025    2.50125
    1.0e12  1.0e12  1.0e12  1.0e12  1.0e12      6.003    6.003     3.0015
    1.0e12  1.0e12  1.0e12  1.0e12  1.0e12      7.0035   7.0035    3.50175
    1.0e12  1.0e12  1.0e12  1.0e12  1.0e12      8.004    8.004     4.002
    1.0e12  1.0e12  1.0e12  1.0e12  1.0e12      9.0045   9.0045    4.50225
    1.0e12  1.0e12  1.0e12  1.0e12  1.0e12  …  10.005   10.005     5.0025
    1.0e12  1.0e12  1.0e12  1.0e12  1.0e12     11.0055  11.0055    5.50275
    1.0e12  1.0e12  1.0e12  1.0e12  1.0e12     12.006   12.006     6.003
    ⋮        

In [111]:
states[1]

0.0

In [98]:
bests[30:40, :]

11×14 Matrix{Float64}:
 -1.0  -1.0   1.0  1.0  1.0  1.0  1.0  1.0  1.0   6.0   6.0   6.0   6.0   6.0
 -1.0  -1.0  -1.0  1.0  1.0  1.0  1.0  1.0  1.0   8.0   8.0   8.0   8.0   8.0
 -1.0  -1.0   1.0  1.0  1.0  1.0  1.0  1.0  1.0  10.0  10.0  10.0  10.0  10.0
 -1.0  -1.0  -1.0  1.0  1.0  1.0  1.0  1.0  1.0  12.0  12.0  12.0  12.0  12.0
 -1.0  -1.0   1.0  1.0  1.0  1.0  1.0  1.0  1.0  14.0  14.0  14.0  14.0  14.0
 -1.0  -1.0   1.0  1.0  1.0  1.0  1.0  1.0  1.0  16.0  16.0  16.0  16.0  16.0
 -1.0  -1.0   1.0  1.0  1.0  1.0  1.0  1.0  1.0  18.0  18.0  18.0  18.0  18.0
 -1.0  -1.0   2.0  2.0  2.0  2.0  2.0  2.0  2.0  20.0  20.0  20.0  20.0  20.0
 -1.0  -1.0   3.0  3.0  3.0  3.0  3.0  3.0  3.0  22.0  22.0  22.0  22.0  22.0
 -1.0  -1.0   4.0  4.0  4.0  4.0  4.0  4.0  4.0  24.0  24.0  24.0  24.0  24.0
 -1.0  -1.0   5.0  5.0  5.0  5.0  5.0  5.0  5.0  26.0  26.0  26.0  26.0  26.0

In [109]:
bests[1, 4]

1.0

In [110]:
f[1, 4]

1-element Vector{Any}:
 (1, 0)

In [58]:
avail_nodes

Dict{Int64, Vector{Int64}} with 15 entries:
  5  => [1, 2, 3, 4, 5, 6, 7, 8, 9, 10  …  37, 38, 39, 40, 41, 42, 43, 44, 45, …
  12 => [1, 2, 3, 4, 5, 6, 7, 8, 9, 10  …  99, 100, 101, 102, 103, 104, 105, 10…
  8  => [1, 2, 3, 4, 5, 6, 7, 8, 9, 10  …  34, 35, 36, 37, 38, 39, 40, 41, 42, …
  1  => [152]
  6  => [1, 2, 3, 4, 5, 6, 7, 8, 9, 10  …  36, 37, 38, 39, 40, 41, 42, 43, 44, …
  11 => [1, 2, 3, 4, 5, 6, 7, 8, 9, 10  …  69, 70, 71, 72, 73, 74, 75, 76, 77, …
  9  => [1, 2, 3, 4, 5, 6, 7, 8, 9, 10  …  33, 34, 35, 36, 37, 38, 39, 40, 41, …
  14 => [1, 2, 3, 4, 5, 6, 7, 8, 9, 10  …  125, 126, 127, 128, 129, 130, 131, 1…
  3  => [16, 17, 18, 19, 20, 21, 22, 23, 24, 25  …  39, 40, 41, 42, 43, 44, 45,…
  7  => [1, 2, 3, 4, 5, 6, 7, 8, 9, 10  …  35, 36, 37, 38, 39, 40, 41, 42, 43, …
  4  => [1, 2, 3, 4, 5, 6, 7, 8, 9, 10  …  38, 39, 40, 41, 42, 43, 44, 45, 46, …
  13 => [1, 2, 3, 4, 5, 6, 7, 8, 9, 10  …  112, 113, 114, 115, 116, 117, 118, 1…
  15 => []
  2  => [33, 34, 36, 38, 39, 41, 43, 44,

In [57]:
costs

152×15 Matrix{Float64}:
 1.0e12  1.0e12  1.0e12  1.0e12  …  1.0e12  1.0e12  1.0e12     0.0
 1.0e12  1.0e12  1.0e12  1.0e12     1.0e12  1.0e12  1.0e12     5.05051
 1.0e12  1.0e12  1.0e12  1.0e12     1.0e12  1.0e12  1.0e12    10.101
 1.0e12  1.0e12  1.0e12  1.0e12     1.0e12  1.0e12  1.0e12    15.1515
 1.0e12  1.0e12  1.0e12  1.0e12     1.0e12  1.0e12  1.0e12    20.202
 1.0e12  1.0e12  1.0e12  1.0e12  …  1.0e12  1.0e12  1.0e12    25.2525
 1.0e12  1.0e12  1.0e12  1.0e12     1.0e12  1.0e12  1.0e12    30.303
 1.0e12  1.0e12  1.0e12  1.0e12     1.0e12  1.0e12  1.0e12    35.3535
 1.0e12  1.0e12  1.0e12  1.0e12     1.0e12  1.0e12  1.0e12    40.404
 1.0e12  1.0e12  1.0e12  1.0e12     1.0e12  1.0e12  1.0e12    45.4545
 1.0e12  1.0e12  1.0e12  1.0e12  …  1.0e12  1.0e12  1.0e12    50.5051
 1.0e12  1.0e12  1.0e12  1.0e12     1.0e12  1.0e12  1.0e12    55.5556
 1.0e12  1.0e12  1.0e12  1.0e12     1.0e12  1.0e12  1.0e12    60.6061
 ⋮                               ⋱                          
 1.0e12  1.

In [820]:
state_costs = states / 2

152-element Vector{Float64}:
    0.0
    5.050505050505051
   10.101010101010102
   15.151515151515152
   20.202020202020204
   25.252525252525253
   30.303030303030305
   35.35353535353535
   40.40404040404041
   45.45454545454545
   50.505050505050505
   55.55555555555555
   60.60606060606061
    ⋮
 4857.402353020311
 5134.294199248329
 5426.968391216569
 5736.324494731868
 6063.313349261811
 6408.939990442711
 6774.266739165433
 7160.416466733641
 7568.576046130221
 8000.000000000001
    4.9999995e6
  250.0

In [807]:
@time a = unique(reduce(vcat, [[j[1] for j in f[i, 14]] for i in 1:size(f)[1] if isassigned(f, i, 14)]));

  0.107884 seconds (199.01 k allocations: 11.685 MiB, 98.43% compilation time)


In [ ]:
function state_entrance_fixed_cost

In [561]:
function get_cost(fire_arc, states, dual_linking)
    
    cost = 0
    if (fire_arc[3] == 1) | (fire_arc[3] == NUM_TIME_PERIODS) 
        cost = states[fire_arc[4]] / 2
    else
        cost = states[fire_arc[4]]
    end
    
    cost += fire_arc[5] * dual_linking[fire_arc[1], fire_arc[3]]
    
    return cost
end

get_cost (generic function with 2 methods)

In [751]:
f

152×15 Matrix{Vector}:
 #undef  …     Any[(1, 0), (2, 0), (4, 2), (7, 4), (3, 1), (6, 3), (9, 6), (5, 3), (8, 5), (10, 7), (13, 9), (11, 7), (14, 10), (12, 8)]
 #undef        Any[(4, 1), (9, 5), (5, 2), (10, 6), (14, 9)]
 #undef        Any[(7, 3), (3, 0), (8, 4), (13, 8), (12, 7), (15, 10)]
 #undef        Any[(6, 2), (5, 1), (10, 5), (11, 6), (15, 9)]
 #undef        Any[(4, 0), (9, 4), (8, 3), (13, 7), (14, 8), (16, 10)]
 #undef  …     Any[(7, 2), (6, 1), (11, 5), (16, 9), (12, 6)]
 #undef        Any[(9, 3), (5, 0), (10, 4), (14, 7), (15, 8), (17, 10)]
 #undef        Any[(7, 1), (8, 2), (13, 6), (12, 5), (17, 9)]
 #undef        Any[(6, 0), (10, 3), (11, 4), (16, 8), (15, 7), (18, 10)]
 #undef        Any[(9, 2), (8, 1), (13, 5), (14, 6), (18, 9)]
 #undef  …     Any[(7, 0), (11, 3), (16, 7), (19, 10), (12, 4), (17, 8)]
 #undef        Any[(9, 1), (10, 2), (14, 5), (19, 9), (15, 6)]
 #undef        Any[(8, 0), (13, 4), (12, 3), (17, 7), (18, 8), (20, 10)]
   ⋮     ⋱  
 #undef  …     Any[(12

In [ ]:
# generate arcs
states_appended = copy(all_states)
push!(states_appended, start_perims[g])
s = length(all_states)
crews_needed = Array{Vector}(undef, s + 1, NUM_TIME_PERIODS + 1)
curr_time = 1
state_name = 0
next_to_check = [s + 1]

while curr_time < NUM_TIME_PERIODS + 1

    to_check = copy(next_to_check)
    next_to_check = []

    for check in to_check
        curr_state = states_appended[check]

        if check != s
            edges = get_alphas(curr_state, fire_progs[curr_time], all_states)
        else
            edges = [(s, 0)]
        end

        for edge in edges
            crews_needed[check, curr_time] = edges
        end
        next_to_check = vcat(next_to_check, [edges[i][1] for i in 1:length(edges) if ~(edges[i][1] in next_to_check)])
    end
    curr_time += 1
end

In [623]:
states[22]

107.6923076923077

In [624]:
e["ceiling"]

2-element Vector{Any}:
 (50, 5)
 (51, 0)

In [625]:
e["floor"]

2-element Vector{Any}:
 (50, 4)
 (51, 0)

In [311]:
function get_alphas(state, prog, sorted_states)
    
    min_state_val = new_perim(state, prog, NUM_CREWS, LINE_PER_CREW)
    max_state_val = new_perim(state, prog, 0, LINE_PER_CREW)
    min_state_ix = searchsorted(sorted_states, min_state_val).start
    max_state_ix = searchsorted(sorted_states, max_state_val).start
    
    edges = []
    for state_ix in min_state_ix:max_state_ix
        push!(edges, (state_ix, get_crews_needed_for_transition(state, 
                                                                 sorted_states[state_ix], 
                                                                 prog,
                                                                 LINE_PER_CREW,
                                                                 "ceiling")
                       ))
    end
    
    return edges

end

get_alphas (generic function with 1 method)

In [562]:
function generate_fire_arcs(g, progs, perims)
    
    # get the progressions for this specific fire
    fire_progs = progs[g, :]
    
    no_supp = [perims[g]]
    for i in 1:NUM_TIME_PERIODS
        push!(no_supp, no_supp[i] * progs[i])
    end

    aggressive_precision = 15
    num_aggressive_states = convert(Int, round(perims[g] * 2 / aggressive_precision))
    num_passive_states = 30

    aggressive_states = LinRange(0, num_aggressive_states * aggressive_precision, num_aggressive_states)
    passive_states = exp.(LinRange(log(num_aggressive_states * aggressive_precision+ 1), maximum(log.(no_supp .+ 1)), num_passive_states + 1))
    passive_states = passive_states[2:num_passive_states+1] .- 1
    all_states = vcat(aggressive_states, passive_states)
    all_states = vcat(all_states, 9999999)

    # generate arcs
    states_appended = copy(all_states)
    push!(states_appended, start_perims[g])
    s = length(all_states)
    crews_needed = Array{Vector}(undef, s + 1, NUM_TIME_PERIODS + 1)
    curr_time = 1
    state_name = 0
    next_to_check = [s + 1]

    while curr_time < NUM_TIME_PERIODS + 1

        to_check = copy(next_to_check)
        next_to_check = []

        for check in to_check
            curr_state = states_appended[check]

            if check != s
                edges = get_alphas(curr_state, fire_progs[curr_time], all_states)
            else
                edges = [(s, 0)]
            end

            for edge in edges
                crews_needed[check, curr_time] = edges
            end
            next_to_check = vcat(next_to_check, [edges[i][1] for i in 1:length(edges) if ~(edges[i][1] in next_to_check)])
        end
        curr_time += 1
    end
    
    visitable = [(i,j) for i in 1:size(crews_needed)[1], j in 1:size(crews_needed)[2] if isassigned(crews_needed, i, j)]
    
    edges = []

    for (i, j) in visitable
        push!(edges, copy(reduce(hcat, [[i, j, a[1], a[2]] for a in crews_needed[i, j]])'))
    end
    
    arc_array = reduce(vcat, edges)
    
    # add crew to front
    arc_array = hcat(convert.(Int, zeros(length(arc_array[:, 1]))) .+ g, arc_array)
    
    return crews_needed, arc_array, states_appended
end

generate_fire_arcs (generic function with 1 method)

In [59]:
function new_perim(old_perim, prog, num_crews, line_per_crew)
    
    line = line_per_crew * num_crews
    return (old_perim - line/2) * prog - line/2

end

new_perim (generic function with 1 method)

In [194]:
function get_crews_needed_for_transition(state_1, state_2, prog, line_per_crew, round_type)
    
    crews = 2 / line_per_crew * (prog * state_1 - state_2) / (1 + prog)
    
    if round_type == "nearest"
        crews = convert(Int, round(crews))
    
    elseif round_type == "ceiling"
        crews = convert(Int, ceil(crews - 0.0001))
    end
    
    return max(crews, 0) 
    
end 

get_crews_needed_for_transition (generic function with 2 methods)

In [563]:
graph, fire_arc_arr, fire_states = generate_fire_arcs(2, progressions, start_perims);

In [560]:
get_cost(fire_arc_arr[i, :] for i in length())

5565×5 Matrix{Int64}:
 2  99   1  23  10
 2  99   1  24   9
 2  99   1  25   8
 2  99   1  26   8
 2  99   1  27   7
 2  99   1  28   6
 2  99   1  29   5
 2  99   1  30   4
 2  99   1  31   3
 2  99   1  32   2
 2  99   1  33   1
 2  99   1  34   0
 2  23   2  12  10
 ⋮              
 2  71  14  76   0
 2  72  14  77   0
 2  73  14  78   0
 2  74  14  79   0
 2  75  14  80   0
 2  76  14  81   0
 2  77  14  82   0
 2  78  14  83   0
 2  79  14  84   0
 2  80  14  85   0
 2  81  14  86   0
 2  82  14  87   0

In [551]:
a[size(a)[1]

(99, 15)

In [554]:
graph = a;

In [555]:
current_node = (size(graph)[1], 1)

(99, 1)

In [559]:
graph[current_node...]

12-element Vector{Any}:
 (23, 10)
 (24, 9)
 (25, 8)
 (26, 8)
 (27, 7)
 (28, 6)
 (29, 5)
 (30, 4)
 (31, 3)
 (32, 2)
 (33, 1)
 (34, 0)

In [529]:
rho

2273×4 Matrix{Int64}:
 45   1   1  10
 45   1   2  10
 45   1   3   9
 45   1   4   9
 45   1   5   8
 45   1   6   8
 45   1   7   8
 45   1   8   7
 45   1   9   7
 45   1  10   6
 45   1  11   6
 45   1  12   6
 45   1  13   5
  ⋮          
 33  14  33   0
 34  14  34   0
 35  14  35   0
 36  14  36   0
 37  14  37   0
 38  14  38   0
 39  14  39   0
 40  14  40   0
 41  14  41   0
 42  14  42   0
 43  14  43   0
 44  14  44   0

In [493]:
g = 3
fire_progs = progressions[g, :]

no_supp = [start_perims[g]]
for i in 1:NUM_TIME_PERIODS
    push!(no_supp, no_supp[i] * fire_progs[i])
end


In [448]:
g = 3
fire_progs = progressions[g, :]


aggressive_precision = 15
num_aggressive_states = convert(Int, round(start_perims[g] * 2 / aggressive_precision))
num_passive_states = 30

aggressive_states = LinRange(0, num_aggressive_states * aggressive_precision, num_aggressive_states)
passive_states = exp.(LinRange(log(num_aggressive_states * aggressive_precision+ 1), maximum(log.(states .+ 1)), num_passive_states + 1))
passive_states = passive_states[2:num_passive_states+1] .- 1
all_states = vcat(aggressive_states, passive_states)
all_states = vcat(all_states, 9999999)

# generate arcs
states_appended = copy(all_states)
push!(states_appended, start_perims[g])
s = length(all_states)
crews_needed = Array{Vector}(undef, s + 1, NUM_TIME_PERIODS + 1)
curr_time = 1
state_name = 0
next_to_check = [s + 1]

while curr_time < 15
    
    to_check = copy(next_to_check)
    next_to_check = []
    
    for check in to_check
        curr_state = states_appended[check]
        
        if check != s
            edges = get_alphas(curr_state, fire_progs[curr_time], all_states)
        else
            edges = [(s, 0)]
        end
        
        for edge in edges
            crews_needed[check, curr_time] = edges
        end
        next_to_check = vcat(next_to_check, [edges[i][1] for i in 1:length(edges) if ~(edges[i][1] in next_to_check)])
    end
    curr_time += 1
end

In [454]:
visitable = [(i,j) for i in 1:size(crews_needed)[1], j in 1:size(crews_needed)[2] if isassigned(crews_needed, i, j)];

In [480]:
edges = []

for (i, j) in visitable
    push!(edges, copy(reduce(hcat, [[i, j, a[1], a[2]] for a in crews_needed[i, j]])'))
end

fire_arcs = reduce(vcat, edges)

2291×4 Matrix{Int64}:
 45   1   1  10
 45   1   2  10
 45   1   3   9
 45   1   4   9
 45   1   5   8
 45   1   6   8
 45   1   7   8
 45   1   8   7
 45   1   9   7
 45   1  10   6
 45   1  11   6
 45   1  12   6
 45   1  13   5
  ⋮          
 34  14  33  10
 34  14  34   0
 35  14  35   0
 36  14  36   0
 37  14  37   0
 38  14  38   0
 39  14  39   0
 40  14  40   0
 41  14  41   0
 42  14  42   0
 43  14  43   0
 44  14  44   0

In [487]:
arc_costs = [get_cost(fire_arcs[i, :], states_appended) for i in 1:length(fire_arcs[:, 1])];

In [ ]:
outs = []

In [481]:
s+1

45

In [482]:
states_appended

45-element Vector{Float64}:
     0.0
    16.25
    32.5
    48.75
    65.0
    81.25
    97.5
   113.75000000000001
   130.0
   146.25
   162.5
   178.75
   195.0
     ⋮
  4270.494082548608
  4945.630954527102
  5727.47751334962
  6632.899905332529
  7681.430078744905
  8895.687130802577
 10301.865251240823
 11930.298788479486
 13816.11662809794
 16000.000000000002
     9.999999e6
   100.0

In [475]:
edgea

14×4 Matrix{Int64}:
 9  2   7  10
 9  2   8  10
 9  2   9  10
 9  2  10   9
 9  2  11   9
 9  2  12   9
 9  2  13   8
 9  2  14   7
 9  2  15   7
 9  2  16   6
 9  2  17   4
 9  2  18   3
 9  2  19   2
 9  2  20   0

In [453]:
crews_needed[45, 1]

18-element Vector{Any}:
 (1, 10)
 (2, 10)
 (3, 9)
 (4, 9)
 (5, 8)
 (6, 8)
 (7, 8)
 (8, 7)
 (9, 7)
 (10, 6)
 (11, 6)
 (12, 6)
 (13, 5)
 (14, 5)
 (15, 4)
 (16, 3)
 (17, 2)
 (18, 0)

In [411]:
sum(crews_needed.!= -1)

LoadError: UndefRefError: access to undefined reference

In [387]:
start_perims[g]

500.0

In [388]:
fire_progs

14-element Vector{Float64}:
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 2.0
 2.0
 2.0
 2.0
 2.0

In [310]:
all_states[38]

8677.520378070449

In [307]:
crews_needed[38, 30:41, 14]

12-element Vector{Float64}:
 -1.0
 -1.0
 -1.0
 -1.0
 -1.0
 -1.0
 -1.0
 -1.0
 -1.0
 -1.0
 -1.0
 -1.0

In [160]:
get_crews_needed_for_transition(curr_state, min_state_val, fire_progs[curr_time], LINE_PER_CREW)

10.0

In [148]:
min_state_ix[18]

LoadError: BoundsError: attempt to access 0-element UnitRange{Int64} at index [18]

In [ ]:
function discretize(fire_model_type, fire_model_data, num_total_crews)

In [78]:
exp.(LinRange(0, minimum(log.(states .+ 1)), 10)) .- 1

10-element Vector{Float64}:
   0.0
   0.9951794800481719
   2.9807411576052933
   6.942293073037287
  14.84630016385273
  30.616212921602965
  62.080019258016065
 124.85596002463721
 250.10522888291908
 499.9999999999999

In [57]:
num_states = 10

10

In [58]:
LinRange(0, 1500, num_states)

10-element LinRange{Float64, Int64}:
 0.0,166.667,333.333,500.0,666.667,833.333,1000.0,1166.67,1333.33,1500.0

In [ ]:
struct FireProgression
    
    states::Vector

In [35]:
states

15-element Vector{Float64}:
 1000.0
    1.5
    0.0
    0.0
    0.0
    0.0
    0.0
    0.0
    0.0
    0.0
    0.0
    0.0
    0.0
    0.0
    0.0

In [145]:
mp = Nothing

Nothing

In [153]:
max_plans = 10000
col_gen_data = initialize_column_generation(A, arc_costs, c_data, progressions, start_perims, max_plans)
current_num_routes = copy(col_gen_data.routes.routes_per_crew)
current_num_plans = copy(col_gen_data.suppression_plans.plans_per_fire)

In [161]:
for gamma in [0, 1, 10, 100, 1000, 10000]

    max_plans = 10000
    col_gen_data = initialize_column_generation(A, arc_costs, c_data, progressions, start_perims, max_plans)
    current_num_routes = copy(col_gen_data.routes.routes_per_crew)
    current_num_plans = copy(col_gen_data.suppression_plans.plans_per_fire)

    objs = []
    max_iters = 200
    n_iters = 0
    opt = false
    tot_time = 0

    while (~opt) & (n_iters < max_iters)

        n_iters += 1

        tot_time += @elapsed mp = run_CG_step(col_gen_data, A, arc_costs, g_data, r_data, rotation_order, gamma)
        push!(objs, objective_value(mp["m"]))

        next_num_routes = col_gen_data.routes.routes_per_crew
        next_num_plans = col_gen_data.suppression_plans.plans_per_fire

        if (sum(next_num_routes) == sum(current_num_routes)) & (sum(next_num_plans) == sum(current_num_plans))
            opt = true
        end

        current_num_routes = copy(next_num_routes)
        current_num_plans = copy(next_num_plans)

    end
    m, p, d, z, q, oor = full_formulation(true, r_data, c_data, rotation_order, arc_costs, 
                              progressions, start_perims, BETA, gamma)
    optimize!(m)
    
    m2, p, d, z, q, oor = full_formulation(false, r_data, c_data, rotation_order, arc_costs, 
                              progressions, start_perims, BETA, 0)
    optimize!(m2)
    
    pb = master_problem(col_gen_data.routes, col_gen_data.suppression_plans, r_data, rotation_order, gamma, true)
    optimize!(pb["m"])
    pct_gap_30 = 100 * (objs[30] / objs[length(objs)] - 1)

    pb_true_gap = 100 * (objective_value(pb["m"]) / objective_value(m) - 1)
    cg_lr_gap = 100 * (objs[length(objs)] / objective_value(m2) - 1)
    
    println(join([gamma, n_iters, tot_time, maximum(current_num_routes), pct_gap_30, pb_true_gap, cg_lr_gap], ' '))
end

0.0 67.0 8.3403557 61.0 0.1137069451359718 46.46200837797869 0.573370980375687
1.0 45.0 5.3362232999999994 41.0 0.10706181654243085 97.9185179287727 0.5712305103596549
10.0 200.0 49.64953460000002 198.0 0.15872186006808953 81.01399513080439 0.5728744706371858
100.0 200.0 55.106949400000026 192.0 0.10306849747885316 55.11832148152911 0.5776160310800194
1000.0 200.0 58.466874100000005 182.0 0.07140729942176627 72.23527377677108 0.5849664010603606
10000.0 195.0 50.829066699999984 111.0 0.025047297145008862 72.68448077601126 0.5853636431418563


In [110]:
a = zeros(NUM_FIRES, NUM_TIME_PERIODS)
for g=1:NUM_FIRES
    for t=1:NUM_TIME_PERIODS
        for c=1:NUM_CREWS
            for r=1:col_gen_data.routes.routes_per_crew[c]-1
                a[g, t] += value(pb["route"][c, r]) * col_gen_data.routes.fires_fought[c, r, g, t] 
            end
        end
    end
end

In [111]:
a

3×14 Matrix{Float64}:
  0.0   0.0   0.0  0.0  2.0  1.0  0.0  3.0  8.0  0.0  0.0  0.0  0.0  0.0
  0.0  10.0  10.0  7.0  4.0  4.0  9.0  2.0  0.0  1.0  0.0  0.0  0.0  0.0
 10.0   0.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0

In [105]:
n_iters

67

In [106]:
objective_value(mp["m"])

1.1510952749095825e6

In [107]:
current_num_routes

10-element Vector{Int64}:
 43
 46
 49
 39
 47
 53
 40
 44
 61
 44

In [74]:
mp = run_CG_step(col_gen_data, A, arc_costs, g_data, r_data, rotation_order, 1000)
objective_value(mp["m"])

([35, 38, 34, 34, 34, 35, 33, 35, 38, 34], [40, 39, 12], 1.1512662246566217e6)

In [260]:
routes.routes_per_crew, suppression_plans.plans_per_fire, objective_value(mp["m"])

([42, 36, 40, 35, 40, 40, 39, 32, 55, 43], [55, 34, 12], 1.1512388539353136e6)

In [309]:
a = zeros(NUM_FIRES, NUM_TIME_PERIODS)
for g=1:NUM_FIRES
    for t=1:NUM_TIME_PERIODS
        for c=1:NUM_CREWS
            for r=1:routes.routes_per_crew[c]-1
                a[g, t] += value(mp["route"][c, r]) * routes.fires_fought[c, r, g, t] 
            end
        end
    end
end

In [308]:
for c=1:NUM_CREWS
    for r=1:routes.routes_per_crew[c]-1
        if value(mp["route"][c, r]) > 0.001
            println((c, r))
        end
    end
end

(1, 23)
(2, 23)
(2, 28)
(3, 35)
(4, 23)
(4, 26)
(5, 27)
(6, 23)
(7, 23)
(8, 21)
(9, 26)
(9, 28)
(10, 33)
(10, 38)


In [316]:
pb = master_problem(routes, suppression_plans, r_data, rotation_order, 10000, true)
optimize!(pb["m"])
objective_value(pb["m"])

2.00453851178113e6

In [318]:
a = zeros(NUM_FIRES, NUM_TIME_PERIODS)
for g=1:NUM_FIRES
    for t=1:NUM_TIME_PERIODS
        for c=1:NUM_CREWS
            for r=1:routes.routes_per_crew[c]-1
                a[g, t] += value(pb["route"][c, r]) * routes.fires_fought[c, r, g, t] 
            end
        end
    end
end

In [321]:
sum(value.(pb["q"]))

2.0

In [319]:
a

3×14 Matrix{Float64}:
 0.0  0.0   0.0   0.0  0.0  1.0  1.0  2.0  1.0  1.0  1.0  0.0  0.0  0.0
 0.0  3.0  10.0  10.0  9.0  1.0  0.0  0.0  1.0  1.0  0.0  0.0  0.0  0.0
 9.0  7.0   0.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0

In [200]:
a = zeros(NUM_FIRES, NUM_TIME_PERIODS)
for g=1:NUM_FIRES
    for t=1:NUM_TIME_PERIODS
        for 
        a[g, t] += sum(value(mp["route"][c, :]) * routes.fires_fought[c, r, g, t] for c=1:NUM_CREWS, r=1:routes.routes_per_crew[c])
    end
end

LoadError: ArgumentError: Indexing with `:` is not supported by Containers.SparseAxisArray

In [864]:
sum(routes.fires_fought[:, 1, :, :])

0

In [1111]:
eta

2-dimensional DenseAxisArray{Float64,2,...} with index sets:
    Dimension 1, Base.OneTo(10)
    Dimension 2, 0:14
And data, a 10×15 Matrix{Float64}:
 0.0  0.0  0.0    0.0    0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0    0.0    0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0    0.0    0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0    0.0    0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0    0.0    0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0    0.0    0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0    0.0    0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0    0.0    0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0    0.0    0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  501.153  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0

In [114]:
routes.out_of_reg[:, 1, :]

10×15 BitMatrix:
 1  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 1  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 1  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 1  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  1  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  1  0  0  0  0  0  1  0  0  0  0  0  0  0
 0  1  0  0  0  0  0  1  0  0  0  1  0  1  0
 0  0  1  0  0  0  1  0  0  0  0  0  0  0  0
 0  0  0  1  0  0  0  0  0  0  0  0  1  0  0

In [42]:
crew_1_arcs = [i for i=1:length(A[:, 1]) if (value(z[i]) > 0.5) & (A[i, 1] == 1)];
cost, ff, oor = get_route_stats(crew_1_arcs, A, arc_costs);

In [170]:
value.(l) / LINE_PER_CREW

3×14 Matrix{Float64}:
 0.0  0.0  0.0  0.0  3.0  7.0  8.0         8.0  8.0  5.0  3.0  3.0  4.0  4.0
 0.0  7.0  9.0  9.0  4.0  0.0  4.11765e-7  0.0  0.0  1.0  1.0  2.0  0.0  3.0
 9.0  2.0  0.0  0.0  0.0  0.0  0.0         0.0  0.0  0.0  1.0  1.0  1.0  1.0

In [34]:
value.(l) / LINE_PER_CREW

3×14 Matrix{Float64}:
 0.0  0.0  0.0  0.0  3.0  7.0  8.0  8.0  8.0  6.0  2.0  2.0  2.0  6.0
 0.0  7.0  9.0  9.0  4.0  0.0  0.0  0.0  0.0  0.0  2.0  1.0  2.0  0.0
 9.0  2.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0

In [33]:
(sum(get_arc_costs(global_data, A2, Dict("cost_per_mile"=>10, "rest_violation" => ones(NUM_CREWS, NUM_TIME_PERIODS)))) - 
    sum(get_arc_costs(global_data, A2, Dict("cost_per_mile"=>10))))

2380.0

In [9]:
function get_out_of_region_stats(region, arcs_used, region_data)
    """
    """
    
    # restrict to the arcs that exited the given region
    out_of_region_ixs = [i for i in length(arcs_used[:, 1]) if arcs_used[i, 10] == region]
    out_of_region_arcs = arcs_used[out_of_region_ixs, :]
    
    # get the crews associated with this region
    crews = [i for i in 1:length(region_data.crew_regions) if region_data.crew_regions[i] == region]
    
    # initialize output array of indicator variables for crews exiting region
    out_array = zeros(length(crews), NUM_TIME_PERIODS)
    
    # for each crew in the region
    for i in 1:length(crews)
        
        # restrict to the arcs involving this crew
        crew_ixs = [j for j in length(out_of_region_arcs[:, 1]) if arcs_used[j, 1] == crews[i]]
        crew_arcs = out_of_region_arcs[crew_ixs, :]
        
        # get the times of rotation
        rotation_times = [t in crew_arcs[:, 6] ? 1 : 0 for t in 1:NUM_TIME_PERIODS]
        
        # update output for crew
        out_array[i, :] = rotation_times
        
    end
    
    return out_array
end

LoadError: syntax: incomplete: "function" at In[9]:1 requires end